# Using Weights & Biases with Tune

(tune-wandb-ref)=

[Weights & Biases](https://www.wandb.ai/) (Wandb) is a tool for experiment
tracking, model optimizaton, and dataset versioning. It is very popular
in the machine learning and data science community for its superb visualization
tools.

```{image} /images/wandb_logo_full.png
:align: center
:alt: Weights & Biases
:height: 80px
:target: https://www.wandb.ai/
```

Ray Tune currently offers two lightweight integrations for Weights & Biases.
One is the {ref}`WandbLoggerCallback <tune-wandb-logger>`, which automatically logs
metrics reported to Tune to the Wandb API.

The other one is the {ref}`@wandb_mixin <tune-wandb-mixin>` decorator, which can be
used with the function API. It automatically
initializes the Wandb API with Tune's training information. You can just use the
Wandb API like you would normally do, e.g. using `wandb.log()` to log your training
process.

```{contents}
:backlinks: none
:local: true
```

## Running A Weights & Biases Example

In the following example we're going to use both of the above methods, namely the `WandbLoggerCallback` and
the `wandb_mixin` decorator to log metrics.
Let's start with a few crucial imports:

In [ ]:
import numpy as np
import wandb

from ray import tune
from ray.tune import Trainable
from ray.tune.integration.wandb import (
    WandbLoggerCallback,
    WandbTrainableMixin,
    wandb_mixin,
)

Next, let's define an easy `objective` function (a Tune `Trainable`) that reports a random loss to Tune.
The objective function itself is not important for this example, since we want to focus on the Weights & Biases
integration primarily.

In [ ]:
def objective(config, checkpoint_dir=None):
    for i in range(30):
        loss = config["mean"] + config["sd"] * np.random.randn()
        tune.report(loss=loss)

Given that you provide an `api_key_file` pointing to your Weights & Biases API key, you cna define a
simple grid-search Tune run using the `WandbLoggerCallback` as follows:

In [ ]:
def tune_function(api_key_file):
    """Example for using a WandbLoggerCallback with the function API"""
    analysis = tune.run(
        objective,
        metric="loss",
        mode="min",
        config={
            "mean": tune.grid_search([1, 2, 3, 4, 5]),
            "sd": tune.uniform(0.2, 0.8),
        },
        callbacks=[
            WandbLoggerCallback(api_key_file=api_key_file, project="Wandb_example")
        ],
    )
    return analysis.best_config

To use the `wandb_mixin` decorator, you can simply decorate the objective function from earlier.
Note that we also use `wandb.log(...)` to log the `loss` to Weights & Biases as a dictionary.
Otherwise, the decorated version of our objective is identical to its original.

In [ ]:
@wandb_mixin
def decorated_objective(config, checkpoint_dir=None):
    for i in range(30):
        loss = config["mean"] + config["sd"] * np.random.randn()
        tune.report(loss=loss)
        wandb.log(dict(loss=loss))

With the `decorated_objective` defined, running a Tune experiment is as simple as providing this objective and
passing the `api_key_file` to the `wandb` key of your Tune `config`:

In [ ]:
def tune_decorated(api_key_file):
    """Example for using the @wandb_mixin decorator with the function API"""
    analysis = tune.run(
        decorated_objective,
        metric="loss",
        mode="min",
        config={
            "mean": tune.grid_search([1, 2, 3, 4, 5]),
            "sd": tune.uniform(0.2, 0.8),
            "wandb": {"api_key_file": api_key_file, "project": "Wandb_example"},
        },
    )
    return analysis.best_config

Finally, you can also define a class-based Tune `Trainable` by using the `WandbTrainableMixin` to define your objective:

In [ ]:
class WandbTrainable(WandbTrainableMixin, Trainable):
    def step(self):
        for i in range(30):
            loss = self.config["mean"] + self.config["sd"] * np.random.randn()
            wandb.log({"loss": loss})
        return {"loss": loss, "done": True}

Running Tune with this `WandbTrainable` works exactly the same as with the function API.
The below `tune_trainable` function differs from `tune_decorated` above only in the first argument we pass to
`tune.run()`:

In [ ]:
def tune_trainable(api_key_file):
    """Example for using a WandTrainableMixin with the class API"""
    analysis = tune.run(
        WandbTrainable,
        metric="loss",
        mode="min",
        config={
            "mean": tune.grid_search([1, 2, 3, 4, 5]),
            "sd": tune.uniform(0.2, 0.8),
            "wandb": {"api_key_file": api_key_file, "project": "Wandb_example"},
        },
    )
    return analysis.best_config

Since you may not have an API key for Wandb, we can _mock_ the Wandb logger and test all three of our training
functions as follows.
If you do have an API key file, make sure to set `mock_api` to `False` and pass in the right `api_key_file` below.

In [ ]:
import tempfile
from unittest.mock import MagicMock

mock_api = True

api_key_file = "~/.wandb_api_key"

if mock_api:
    WandbLoggerCallback._logger_process_cls = MagicMock
    decorated_objective.__mixins__ = tuple()
    WandbTrainable._wandb = MagicMock()
    wandb = MagicMock()  # noqa: F811
    temp_file = tempfile.NamedTemporaryFile()
    temp_file.write(b"1234")
    temp_file.flush()
    api_key_file = temp_file.name

tune_function(api_key_file)
tune_decorated(api_key_file)
tune_trainable(api_key_file)

if mock_api:
    temp_file.close()

This completes our Tune and Wandb walk-through.
In the following sections you can find more details on the API of the Tune-Wandb integration.

## Tune Wandb API Reference

### WandbLoggerCallback

(tune-wandb-logger)=

```{eval-rst}
.. autoclass:: ray.tune.integration.wandb.WandbLoggerCallback
   :noindex:
```

### Wandb-Mixin

(tune-wandb-mixin)=

```{eval-rst}
.. autofunction:: ray.tune.integration.wandb.wandb_mixin
   :noindex:
```